In [1]:
from pyspark.sql import SparkSession

spark = (
    SparkSession.builder.appName("ETL")
    .config("spark.jars", "postgresql-42.6.0.jar,clickhouse-jdbc-0.4.6.jar")
    .getOrCreate()
)

pg_jdbc_url = "jdbc:postgresql://postgres:5432/spark_db"
pg_properties = {
    "user": "spark_user",
    "password": "spark_password",
    "driver": "org.postgresql.Driver",
}

ch_jdbc_url = "jdbc:clickhouse://clickhouse:8123/default"
ch_properties = {
    "driver": "com.clickhouse.jdbc.ClickHouseDriver",
    "user": "custom_user",
    "password": "custom_password",
}

In [2]:
from pyspark.sql.functions import col, lit

df_mock_data = spark.read.jdbc(pg_jdbc_url, "public.mock_data", properties=pg_properties)
df_mock_data.toPandas().head()

,id,customer_first_name,customer_last_name,customer_age,customer_email,customer_country,customer_postal_code,customer_pet_type,customer_pet_name,customer_pet_breed,...,product_reviews,product_release_date,product_expiry_date,supplier_name,supplier_contact,supplier_email,supplier_phone,supplier_address,supplier_city,supplier_country
0,1,Conni,Leydon,63,lswait0@amazon.com,France,77404 CEDEX,cat,Jan,Labrador Retriever,...,360,9/29/2022,2/28/2026,Thoughtbeat,Lenee Swait,lswait0@oracle.com,852-750-6042,Room 1411,Batangafo,Russia
1,2,Alec,Chamberlayne,26,cwalsh1@state.gov,Philippines,9401,dog,Shelia,Labrador Retriever,...,754,5/31/2016,9/11/2029,Eayo,Colet Walsh,cwalsh1@addthis.com,651-123-9474,Suite 4,Labuan,Sweden
2,3,Vaughan,Shapiro,47,jelnough2@xing.com,Madagascar,None,cat,Gunner,Siamese,...,313,12/14/2012,1/5/2024,Gabspot,Jane Elnough,jelnough2@ezinearticles.com,325-676-3913,Room 1133,Veinticinco de Mayo,Portugal
3,4,Bank,Audas,81,lscay3@howstuffworks.com,Portugal,2860-010,bird,Nahum,Labrador Retriever,...,938,11/6/2010,6/3/2025,Jabberstorm,Lorilyn Scay,lscay3@opensource.org,112-474-3398,Room 1500,Saraktash,China
4,5,Anthe,Pletts,72,ctoll4@miibeian.gov.cn,Russia,628389,cat,Brod,Labrador Retriever,...,943,11/2/2018,7/10/2030,Trudeo,Court Toll,ctoll4@usgs.gov,647-688-5659,Apt 1574,Lakatnik,China


# Трансформация данных из исходной модели в снежинку

In [3]:
def write_distinct_to_lookup(df, source_column, target_table, lookup_column_name="name"):
    new_values_df = df.select(col(source_column).alias(lookup_column_name)).distinct().filter(col(lookup_column_name).isNotNull())
    try:
        existing_values_df = spark.read.jdbc(pg_jdbc_url, target_table, properties=pg_properties).select(lookup_column_name)

        values_to_insert_df = new_values_df.join(
            existing_values_df,
            new_values_df[lookup_column_name] == existing_values_df[lookup_column_name],
            "left_anti"
        )
    except Exception as e:
        values_to_insert_df = new_values_df
    
    if values_to_insert_df.count() > 0:
        values_to_insert_df.write.jdbc(url=pg_jdbc_url, table=target_table, mode="append", properties=pg_properties)


# countries
write_distinct_to_lookup(df_mock_data, "customer_country", "countries")
write_distinct_to_lookup(df_mock_data, "seller_country", "countries")
write_distinct_to_lookup(df_mock_data, "store_country", "countries")
write_distinct_to_lookup(df_mock_data, "supplier_country", "countries")

# cities
write_distinct_to_lookup(df_mock_data, "store_city", "cities")
write_distinct_to_lookup(df_mock_data, "supplier_city", "cities")

# pet_types
write_distinct_to_lookup(df_mock_data, "customer_pet_type", "pet_types")

# pet_breeds
write_distinct_to_lookup(df_mock_data, "customer_pet_breed", "pet_breeds")

# pet_categories
write_distinct_to_lookup(df_mock_data, "pet_category", "pet_categories")

# product_names
write_distinct_to_lookup(df_mock_data, "product_name", "product_names")

# product_categories
write_distinct_to_lookup(df_mock_data, "product_category", "product_categories")

# product_brands
write_distinct_to_lookup(df_mock_data, "product_brand", "product_brands")

# product_colors
write_distinct_to_lookup(df_mock_data, "product_color", "product_colors")

# product_sizes
write_distinct_to_lookup(df_mock_data, "product_size", "product_sizes")

# product_materials
write_distinct_to_lookup(df_mock_data, "product_material", "product_materials")

In [4]:
df_countries = spark.read.jdbc(pg_jdbc_url, "countries", properties=pg_properties).select(col("id").alias("country_id"), col("name").alias("country_name"))
df_cities = spark.read.jdbc(pg_jdbc_url, "cities", properties=pg_properties).select(col("id").alias("city_id"), col("name").alias("city_name"))
df_pet_types = spark.read.jdbc(pg_jdbc_url, "pet_types", properties=pg_properties).select(col("id").alias("pet_type_id"), col("name").alias("pet_type_name"))
df_pet_breeds = spark.read.jdbc(pg_jdbc_url, "pet_breeds", properties=pg_properties).select(col("id").alias("pet_breed_id"), col("name").alias("pet_breed_name"))
df_pet_categories = spark.read.jdbc(pg_jdbc_url, "pet_categories", properties=pg_properties).select(col("id").alias("pet_category_id"), col("name").alias("pet_category_name"))
df_product_names = spark.read.jdbc(pg_jdbc_url, "product_names", properties=pg_properties).select(col("id").alias("product_name_id"), col("name").alias("product_name_val"))
df_product_categories = spark.read.jdbc(pg_jdbc_url, "product_categories", properties=pg_properties).select(col("id").alias("product_category_id"), col("name").alias("product_category_val"))
df_product_brands = spark.read.jdbc(pg_jdbc_url, "product_brands", properties=pg_properties).select(col("id").alias("product_brand_id"), col("name").alias("product_brand_val"))
df_product_colors = spark.read.jdbc(pg_jdbc_url, "product_colors", properties=pg_properties).select(col("id").alias("product_color_id"), col("name").alias("product_color_val"))
df_product_sizes = spark.read.jdbc(pg_jdbc_url, "product_sizes", properties=pg_properties).select(col("id").alias("product_size_id"), col("name").alias("product_size_val"))
df_product_materials = spark.read.jdbc(pg_jdbc_url, "product_materials", properties=pg_properties).select(col("id").alias("product_material_id"), col("name").alias("product_material_val"))

In [5]:
# customers
df_customers = (
    df_mock_data.alias("md")
    .join(
        df_countries.alias("c"),
        col("md.customer_country") == col("c.country_name"),
        "left",
    )
    .join(
        df_pet_types.alias("pt"),
        col("md.customer_pet_type") == col("pt.pet_type_name"),
        "left",
    )
    .join(
        df_pet_breeds.alias("pb"),
        col("md.customer_pet_breed") == col("pb.pet_breed_name"),
        "left",
    )
    .join(
        df_pet_categories.alias("pc"),
        col("md.pet_category") == col("pc.pet_category_name"),
        "left",
    )
    .select(
        col("md.customer_first_name").alias("first_name"),
        col("md.customer_last_name").alias("last_name"),
        col("md.customer_age").alias("age"),
        col("md.customer_email").alias("email"),
        col("md.customer_postal_code").alias("postal_code"),
        col("md.customer_pet_name").alias("pet_name"),
        col("pt.pet_type_id").alias("pet_type_id"),
        col("pb.pet_breed_id").alias("pet_breed_id"),
        col("pc.pet_category_id").alias("pet_category_id"),
        col("c.country_id").alias("country_id"),
    )
    .distinct()
)

df_customers.write.jdbc(
    url=pg_jdbc_url, table="customers", mode="append", properties=pg_properties
)

In [6]:
# stores
df_stores = (
    df_mock_data.alias("md")
    .join(
        df_countries.alias("c"),
        col("md.store_country") == col("c.country_name"),
        "left",
    )
    .join(df_cities.alias("ci"), col("md.store_city") == col("ci.city_name"), "left")
    .select(
        col("md.store_name").alias("name"),
        col("md.store_location").alias("location"),
        col("md.store_state").alias("state"),
        col("md.store_phone").alias("phone"),
        col("md.store_email").alias("email"),
        col("c.country_id").alias("country_id"),
        col("ci.city_id").alias("city_id"),
    )
    .distinct()
)

df_stores.write.jdbc(
    url=pg_jdbc_url, table="stores", mode="append", properties=pg_properties
)

In [7]:
# sellers
df_stores = spark.read.jdbc(pg_jdbc_url, "stores", properties=pg_properties)

df_sellers = (
    df_mock_data.alias("md")
    .join(
        df_countries.alias("c"),
        col("md.seller_country") == col("c.country_name"),
        "left",
    )
    .join(
        df_stores.alias("s"),
        col("md.store_email") == col("s.email"),
        "left"
    )
    .select(
        col("md.seller_first_name").alias("first_name"),
        col("md.seller_last_name").alias("last_name"),
        col("md.seller_email").alias("email"),
        col("md.seller_postal_code").alias("postal_code"),
        col("c.country_id").alias("country_id"),
        col("s.id").alias("store_id")
    )
    .distinct()
)

df_sellers.write.jdbc(
    url=pg_jdbc_url, table="sellers", mode="append", properties=pg_properties
)

In [8]:
# suppliers
df_suppliers = (
    df_mock_data.alias("md")
    .join(
        df_countries.alias("c"),
        col("md.supplier_country") == col("c.country_name"),
        "left",
    )
    .join(df_cities.alias("ci"), col("md.supplier_city") == col("ci.city_name"), "left")
    .select(
        col("md.supplier_name").alias("name"),
        col("md.supplier_contact").alias("contact"),
        col("md.supplier_email").alias("email"),
        col("md.supplier_phone").alias("phone"),
        col("md.supplier_address").alias("address"),
        col("c.country_id").alias("country_id"),
        col("ci.city_id").alias("city_id"),
    )
    .distinct()
)

df_suppliers.write.jdbc(
    url=pg_jdbc_url, table="suppliers", mode="append", properties=pg_properties
)

In [9]:
# products
df_suppliers = spark.read.jdbc(pg_jdbc_url, "suppliers", properties=pg_properties)

df_products = (
    df_mock_data.alias("md")
    .join(
        df_product_names.alias("pn"),
        col("md.product_name") == col("pn.product_name_val"),
        "left",
    )
    .join(
        df_product_categories.alias("pcg"),
        col("md.product_category") == col("pcg.product_category_val"),
        "left",
    )
    .join(
        df_product_sizes.alias("ps"),
        col("md.product_size") == col("ps.product_size_val"),
        "left",
    )
    .join(
        df_product_colors.alias("pcol"),
        col("md.product_color") == col("pcol.product_color_val"),
        "left",
    )
    .join(
        df_product_brands.alias("pb"),
        col("md.product_brand") == col("pb.product_brand_val"),
        "left",
    )
    .join(
        df_product_materials.alias("pm"),
        col("md.product_material") == col("pm.product_material_val"),
        "left",
    )
    .join(
        df_suppliers.alias("s"),
        col("md.supplier_name") == col("s.name"),
        "left",
    )
    .select(
        col("md.product_price").alias("price"),
        col("md.product_quantity").alias("quantity"),
        col("md.product_weight").alias("weight"),
        col("md.product_description").alias("description"),
        col("md.product_rating").alias("rating"),
        col("md.product_reviews").alias("reviews"),
        col("md.product_release_date").alias("release_date"),
        col("md.product_expiry_date").alias("expiry_date"),
        col("pn.product_name_id").alias("product_name_id"),
        col("pcg.product_category_id").alias("product_category_id"),
        col("ps.product_size_id").alias("product_size_id"),
        col("pcol.product_color_id").alias("product_color_id"),
        col("pb.product_brand_id").alias("product_brand_id"),
        col("pm.product_material_id").alias("product_material_id"),
        col("s.id").alias("supplier_id"),
    )
    .distinct()
)

df_products.write.jdbc(
    url=pg_jdbc_url, table="products", mode="append", properties=pg_properties
)

In [10]:
df_customers_lookup = spark.read.jdbc(
    pg_jdbc_url, "customers", properties=pg_properties
).select(col("id").alias("customer_id_pk"), col("email").alias("customer_email_pk"))
df_sellers_lookup = spark.read.jdbc(
    pg_jdbc_url, "sellers", properties=pg_properties
).select(col("id").alias("seller_id_pk"), col("email").alias("seller_email_pk"))
df_products_lookup = (
    spark.read.jdbc(pg_jdbc_url, "products", properties=pg_properties)
    .alias("p")
    .join(
        df_product_names.alias("pn_lk"),
        col("p.product_name_id") == col("pn_lk.product_name_id"),
        "inner",
    )
    .select(
        col("p.id").alias("product_id_pk"),
        col("p.price").alias("product_price_pk"),
        col("p.quantity").alias("product_quantity_pk"),
        col("pn_lk.product_name_val").alias("product_name_pk"),
    )
)


df_sales = (
    df_mock_data.alias("md")
    .join(
        df_customers_lookup.alias("cust"),
        col("md.customer_email") == col("cust.customer_email_pk"),
        "left",
    )
    .join(
        df_sellers_lookup.alias("sell"),
        col("md.seller_email") == col("sell.seller_email_pk"),
        "left",
    )
    .join(
        df_products_lookup.alias("prod"),
        (col("md.product_price") == col("prod.product_price_pk"))
        & (col("md.product_quantity") == col("prod.product_quantity_pk"))
        & (col("md.product_name") == col("prod.product_name_pk")),
        "left",
    )
    .select(
        col("md.sale_date").alias("sale_date"),
        col("cust.customer_id_pk").alias("sale_customer_id"),
        col("sell.seller_id_pk").alias("sale_seller_id"),
        col("prod.product_id_pk").alias("sale_product_id"),
        col("md.sale_quantity").alias("sale_quantity"),
        col("md.sale_total_price").alias("sale_total_price"),
    )
)

df_sales.write.jdbc(
    url=pg_jdbc_url, table="sales", mode="append", properties=pg_properties
)

# Отчеты в clickhouse

### Витрина продаж по продуктам Цель: Анализ выручки, количества продаж и популярности продуктов.
 
* Топ-10 самых продаваемых продуктов.
* Общая выручка по категориям продуктов.
* Средний рейтинг и количество отзывов для каждого продукта.

In [11]:
from pyspark.sql.functions import (
    col,
    sum,
    avg,
    count,
    rank,
    dense_rank,
    month,
    year,
    from_unixtime,
    to_date,
    date_format,
)
from pyspark.sql.window import Window

df_sales = spark.read.jdbc(pg_jdbc_url, "sales", properties=pg_properties)
df_customers = spark.read.jdbc(pg_jdbc_url, "customers", properties=pg_properties)
df_products = spark.read.jdbc(pg_jdbc_url, "products", properties=pg_properties)
df_sellers = spark.read.jdbc(pg_jdbc_url, "sellers", properties=pg_properties)
df_stores = spark.read.jdbc(pg_jdbc_url, "stores", properties=pg_properties)
df_suppliers = spark.read.jdbc(pg_jdbc_url, "suppliers", properties=pg_properties)
df_stores_renamed = df_stores.withColumnRenamed("id", "store_id").withColumnRenamed("name", "store_name")
df_sellers_renamed = df_sellers.withColumnRenamed("id", "seller_id")

df_product_names = spark.read.jdbc(
    pg_jdbc_url, "product_names", properties=pg_properties
).select(col("id").alias("product_name_id"), col("name").alias("product_name"))
df_product_categories = spark.read.jdbc(
    pg_jdbc_url, "product_categories", properties=pg_properties
).select(col("id").alias("product_category_id"), col("name").alias("product_category"))
df_countries = spark.read.jdbc(
    pg_jdbc_url, "countries", properties=pg_properties
).select(col("id").alias("country_id"), col("name").alias("country_name"))
df_cities = spark.read.jdbc(pg_jdbc_url, "cities", properties=pg_properties).select(
    col("id").alias("city_id"), col("name").alias("city_name")
)

df_product_sales = (
    df_sales.join(df_products, df_sales.sale_product_id == df_products.id, "inner")
    .join(
        df_product_names,
        df_products.product_name_id == df_product_names.product_name_id,
        "left",
    )
    .join(
        df_product_categories,
        df_products.product_category_id == df_product_categories.product_category_id,
        "left",
    )
)

top_10_products_df = df_product_sales.groupBy("product_name", "product_category").agg(
    sum("sale_total_price").alias("total_revenue"),
    sum("sale_quantity").alias("total_sales_quantity")
)

window_spec_rank_product = Window.orderBy(col("total_revenue").desc())
top_10_products_final = (
    top_10_products_df.withColumn("rank", rank().over(window_spec_rank_product))
    .filter(col("rank") <= 10)
    .drop("rank")
)

top_10_products_final.write.jdbc(
    url=ch_jdbc_url,
    table="top_10_products_report",
    mode="append",
    properties=ch_properties,
)

top_10_products_final.toPandas().head(10)

,product_name,product_category,total_revenue,total_sales_quantity
0,Dog Food,Toy,8.388194e+06,180267
1,Bird Cage,Toy,8.339287e+06,173390
2,Cat Toy,Cage,7.957720e+06,173243
3,Bird Cage,Cage,7.950889e+06,177294
4,Dog Food,Food,7.857492e+06,171988
5,Cat Toy,Toy,7.853075e+06,170663
6,Cat Toy,Food,7.729019e+06,170441
7,Bird Cage,Food,7.715254e+06,161557
8,Dog Food,Cage,7.657714e+06,162261


In [12]:
revenue_by_category_df = df_product_sales.groupBy("product_category").agg(
    sum("sale_total_price").alias("category_total_revenue")
)

revenue_by_category_df.write.jdbc(
    url=ch_jdbc_url,
    table="revenue_by_category_report",
    mode="append",
    properties=ch_properties,
)

revenue_by_category_df.toPandas().head()

,product_category,category_total_revenue
0,Cage,2.356632e+07
1,Food,2.330176e+07
2,Toy,2.458056e+07


In [13]:
product_reviews_ratings_df = df_product_sales.groupBy("product_name", "product_category").agg(
    avg("rating").alias("avg_product_rating"),
    sum("reviews").alias("total_product_reviews")
)

product_reviews_ratings_df.write.jdbc(
    url=ch_jdbc_url,
    table="product_reviews_ratings_report",
    mode="append",
    properties=ch_properties,
)
product_reviews_ratings_df.toPandas().head()

,product_name,product_category,avg_product_rating,total_product_reviews
0,Dog Food,Toy,2.997150,16669825
1,Dog Food,Food,3.020548,15409577
2,Dog Food,Cage,3.058483,14762557
3,Bird Cage,Toy,2.995688,15791066
4,Bird Cage,Cage,3.003047,16585782


### Витрина продаж по клиентам Цель: Анализ покупательского поведения и сегментация клиентов.
* Топ-10 клиентов с наибольшей общей суммой покупок.
* Распределение клиентов по странам.
* Средний чек для каждого клиента.

In [14]:
df_customer_sales = (
    df_sales.alias("s")
    .join(df_customers.alias("c"), col("s.sale_customer_id") == col("c.id"), "inner")
    .join(df_countries.alias("co"), col("c.country_id") == col("co.country_id"), "left")
    .select(
        col("s.sale_total_price"),
        col("s.id").alias("sale_id"),
        col("c.first_name").alias("customer_first_name"),
        col("c.last_name").alias("customer_last_name"),
        col("c.email").alias("customer_email"),
        col("co.country_name").alias("customer_country"),
    )
)

customer_total_spent_df = df_customer_sales.groupBy(
    "customer_first_name", "customer_last_name", "customer_email"
).agg(
    sum("sale_total_price").alias("total_spent")
)

window_spec_rank_customer = Window.orderBy(col("total_spent").desc())
top_10_customers_final = (
    customer_total_spent_df.withColumn("rank", rank().over(window_spec_rank_customer))
    .filter(col("rank") <= 10)
    .drop("rank")
)

top_10_customers_final.write.jdbc(
    url=ch_jdbc_url,
    table="top_10_customers_report",
    mode="append",
    properties=ch_properties,
)

top_10_customers_final.toPandas().head(10)

,customer_first_name,customer_last_name,customer_email,total_spent
0,Fredric,Warnes,cgilderspb@dedecms.com,32784.839355
1,Vidovic,Duffin,rannablefv@samsung.com,32710.259033
2,Mervin,Garmon,cverheydenji@berkeley.edu,32433.720886
3,Vi,Renard,fconerskm@wikia.com,32365.740967
4,Northrup,Ortell,eroscrigg1b@com.com,31876.020081
5,Livvyy,Haacker,scartmelfk@adobe.com,31053.000000
6,Nerti,Fernihough,amilesopqn@rediff.com,30843.779114
7,Dallis,Date,kchristoffelsc8@reference.com,30794.939758
8,Katerine,Screech,bgiacovelli7t@about.com,30714.419678
9,Guenna,Laye,dsollett5q@meetup.com,30445.799194


In [15]:
customers_by_country_df = df_customer_sales.select("customer_email", "customer_country").distinct().groupBy("customer_country").agg(
    count("customer_email").alias("customer_count")
)

customers_by_country_df.write.jdbc(
    url=ch_jdbc_url,
    table="customers_by_country_report",
    mode="append",
    properties=ch_properties,
)

customers_by_country_df.toPandas().head()

,customer_country,customer_count
0,Chad,5
1,Russia,628
2,Paraguay,18
3,Yemen,39
4,U.S. Virgin Islands,1


In [16]:
average_check_by_customer_df = df_customer_sales.groupBy(
    "customer_first_name", "customer_last_name", "customer_email"
).agg(
    sum("sale_total_price").alias("total_spent"),
    count("sale_id").alias("total_orders")
)

average_check_by_customer_final = average_check_by_customer_df.withColumn(
    "average_order_value",
    col("total_spent") / col("total_orders")
).select(
    "customer_first_name",
    "customer_last_name",
    "customer_email",
    "average_order_value"
)
average_check_by_customer_final.write.jdbc(
    url=ch_jdbc_url,
    table="average_check_by_customer_report",
    mode="append",
    properties=ch_properties,
)
average_check_by_customer_final.toPandas().head()

,customer_first_name,customer_last_name,customer_email,average_order_value
0,Mariejeanne,McAnulty,etownbs@1688.com,128.960007
1,Osmund,Moylane,jdelapev@hc360.com,355.440002
2,Cathyleen,Swindley,crosendall18@yahoo.co.jp,387.029999
3,Laurel,McCartney,eleedalr4@cam.ac.uk,136.619995
4,Perry,Edwins,mkardosrn@jimdo.com,214.000000


### Витрина продаж по времени Цель: Анализ сезонности и трендов продаж.
* Месячные и годовые тренды продаж.
* Сравнение выручки за разные периоды.
* Средний размер заказа по месяцам.

In [17]:
df_sales_time = df_sales.withColumn(
    "sale_date_parsed", to_date(col("sale_date"), "M/d/yyyy")
)

df_sales_time = df_sales_time.withColumn(
    "sale_year", year(col("sale_date_parsed"))
).withColumn("sale_month", month(col("sale_date_parsed")))


monthly_yearly_sales_trends_df = df_sales_time.groupBy(
    "sale_year", "sale_month"
).agg(
    sum("sale_total_price").alias("total_revenue"),
    count("id").alias("total_orders"),
).orderBy("sale_year", "sale_month")

monthly_yearly_sales_trends_df.write.jdbc(
    url=ch_jdbc_url,
    table="monthly_yearly_sales_trends_report",
    mode="append",
    properties=ch_properties,
)

monthly_yearly_sales_trends_df.toPandas().head()

,sale_year,sale_month,total_revenue,total_orders
0,2021,1,6.282812e+06,24427
1,2021,2,5.360731e+06,20586
2,2021,3,5.937631e+06,23997
3,2021,4,5.781788e+06,23702
4,2021,5,6.075551e+06,23861


In [18]:
yearly_sales_summary_df = df_sales_time.groupBy(
    "sale_year"
).agg(
    sum("sale_total_price").alias("yearly_total_revenue"),
    count("id").alias("yearly_total_orders"),
).orderBy("sale_year")

yearly_sales_summary_df.write.jdbc(
    url=ch_jdbc_url,
    table="yearly_sales_summary_report",
    mode="append",
    properties=ch_properties,
)

yearly_sales_summary_df.toPandas().head()

,sale_year,yearly_total_revenue,yearly_total_orders
0,2021,7.144864e+07,282189


In [19]:
average_order_value_by_month_df_temp = df_sales_time.groupBy(
    "sale_year", "sale_month"
).agg(
    sum("sale_total_price").alias("total_monthly_revenue"),
    count("id").alias("total_monthly_orders"),
)

average_order_value_by_month_final = average_order_value_by_month_df_temp.withColumn(
    "average_order_value",
    col("total_monthly_revenue") / col("total_monthly_orders")
).select(
    "sale_year",
    "sale_month",
    "average_order_value"
).orderBy("sale_year", "sale_month")

average_order_value_by_month_final.write.jdbc(
    url=ch_jdbc_url,
    table="average_order_value_by_month_report",
    mode="append",
    properties=ch_properties,
)

average_order_value_by_month_final.toPandas().head()

,sale_year,sale_month,average_order_value
0,2021,1,257.207660
1,2021,2,260.406656
2,2021,3,247.432233
3,2021,4,243.936726
4,2021,5,254.622651


### Витрина продаж по магазинам Цель: Анализ эффективности магазинов.
* Топ-5 магазинов с наибольшей выручкой.
* Распределение продаж по городам и странам.
* Средний чек для каждого магазина.

In [20]:
df_store_sales = (
    df_sales.alias("s")
    .join(df_sellers_renamed.alias("sel"), col("s.sale_seller_id") == col("sel.seller_id"), "inner")
    .join(df_stores_renamed.alias("st"), col("sel.store_id") == col("st.store_id"), "inner")
    .join(df_cities.alias("ci"), col("st.city_id") == col("ci.city_id"), "left")
    .join(df_countries.alias("co"), col("st.country_id") == col("co.country_id"), "left")
    .select(
        col("st.store_name"),
        col("ci.city_name"),
        col("co.country_name").alias("store_country"),
        col("s.sale_total_price"),
        col("s.id").alias("sale_id")
    )
)
store_revenue_summary = df_store_sales.groupBy("store_name").agg(
    sum("sale_total_price").alias("total_store_revenue")
)

window_spec_rank_store = Window.orderBy(col("total_store_revenue").desc())
top_5_stores_final = (
    store_revenue_summary.withColumn("rank", rank().over(window_spec_rank_store))
    .filter(col("rank") <= 5)
    .drop("rank")
)

top_5_stores_final.write.jdbc(
    url=ch_jdbc_url,
    table="top_5_stores_report",
    mode="append",
    properties=ch_properties,
)

top_5_stores_final.toPandas().head()

,store_name,total_store_revenue
0,Mynte,508126.769417
1,Quatz,436353.500446
2,Quinu,419202.228113
3,Jayo,413849.111885
4,Katz,380432.343393


In [21]:
city_sales_data = df_store_sales.select(
    col("store_country").alias("location_name"),
    lit("Country").alias("location_type"),
    col("sale_total_price"),
    col("sale_id")
)

sales_by_location_final = city_sales_data.groupBy("location_name", "location_type").agg(
    sum("sale_total_price").alias("total_sales_revenue"),
    count("sale_id").alias("total_sales_count")
)

sales_by_location_final.write.jdbc(
    url=ch_jdbc_url,
    table="sales_by_location_report",
    mode="append",
    properties=ch_properties,
)

sales_by_location_final.toPandas().head()

,location_name,location_type,total_sales_revenue,total_sales_count
0,Qatar,Country,7956.779968,52
1,Burkina Faso,Country,34099.230408,97
2,Saint Kitts and Nevis,Country,20431.669952,75
3,Belarus,Country,235099.299263,965
4,Uzbekistan,Country,139738.239227,552


In [22]:
store_average_check_df = df_store_sales.groupBy("store_name").agg(
    sum("sale_total_price").alias("total_store_revenue"),
    count("sale_id").alias("total_store_orders")
)

average_check_by_store_final = store_average_check_df.withColumn(
    "average_store_order_value",
    col("total_store_revenue") / col("total_store_orders")
).select(
    "store_name",
    "average_store_order_value"
)

average_check_by_store_final.write.jdbc(
    url=ch_jdbc_url,
    table="average_check_by_store_report",
    mode="append",
    properties=ch_properties,
)

average_check_by_store_final.toPandas().head()

,store_name,average_store_order_value
0,Jetwire,256.483166
1,Jaxworks,263.644569
2,Brainlounge,246.884861
3,Reallinks,270.400601
4,Snaptags,262.824886


### Витрина продаж по поставщикам Цель: Анализ эффективности поставщиков.
* Топ-5 поставщиков с наибольшей выручкой.
* Средняя цена товаров от каждого поставщика.
* Распределение продаж по странам поставщиков.

In [23]:
df_supplier_sales = (
    df_sales.alias("s")
    .join(df_products.alias("p"), col("s.sale_product_id") == col("p.id"), "inner")
    .join(
        df_product_names.alias("pn"),
        col("p.product_name_id") == col("pn.product_name_id"),
        "left",
    )
    .join(df_suppliers.alias("sup"), col("p.supplier_id") == col("sup.id"), "left")
    .join(df_countries.alias("co"), col("sup.country_id") == col("co.country_id"), "left")
    .select(
        col("sup.name").alias("supplier_name"),
        col("co.country_name").alias("supplier_country"),
        col("s.sale_total_price"),
        col("p.price").alias("product_unit_price"),
        col("s.id").alias("sale_id")
    )
)

supplier_revenue_df = df_supplier_sales.groupBy("supplier_name", "supplier_country").agg(
    sum("sale_total_price").alias("total_revenue_from_supplier")
)

window_spec_rank_supplier = Window.orderBy(col("total_revenue_from_supplier").desc())
top_5_suppliers_final = (
    supplier_revenue_df.withColumn("rank", rank().over(window_spec_rank_supplier))
    .filter(col("rank") <= 5)
    .drop("rank")
)

top_5_suppliers_final.write.jdbc(
    url=ch_jdbc_url,
    table="top_5_suppliers_report",
    mode="append",
    properties=ch_properties,
)

top_5_suppliers_final.toPandas().head()

,supplier_name,supplier_country,total_revenue_from_supplier
0,Wikizz,China,248216.219860
1,Wikizz,Indonesia,248216.219860
2,Katz,China,245582.699203
3,Livetube,China,191642.880188
4,Jayo,China,158499.440346


In [24]:
avg_price_by_supplier_df = df_supplier_sales.groupBy("supplier_name", "supplier_country").agg(
    avg("product_unit_price").alias("average_product_price_from_supplier")
)

avg_price_by_supplier_df.write.jdbc(
    url=ch_jdbc_url,
    table="avg_product_price_by_supplier_report",
    mode="append",
    properties=ch_properties,
)

avg_price_by_supplier_df.toPandas().head()

,supplier_name,supplier_country,average_product_price_from_supplier
0,Kwideo,France,44.066296
1,Eazzy,China,53.550000
2,Jabbersphere,Czech Republic,46.082045
3,Lajo,Germany,52.257272
4,Zooxo,Russia,47.569143


In [25]:
sales_distribution_by_supplier_country_df = df_supplier_sales.groupBy("supplier_country").agg(
    sum("sale_total_price").alias("total_revenue_from_country")
)

sales_distribution_by_supplier_country_df.write.jdbc(
    url=ch_jdbc_url,
    table="sales_distribution_by_supplier_country_report",
    mode="append",
    properties=ch_properties,
)

sales_distribution_by_supplier_country_df.toPandas().head()

,supplier_country,total_revenue_from_country
0,Chad,4.349300e+04
1,Russia,4.077664e+06
2,Paraguay,1.181418e+05
3,Yemen,3.110932e+05
4,Senegal,7.437471e+04


### Витрина качества продукции Цель: Анализ отзывов и рейтингов товаров.
* Продукты с наивысшим и наименьшим рейтингом.
* Корреляция между рейтингом и объемом продаж.
* Продукты с наибольшим количеством отзывов.

In [26]:
df_product_quality_base = (
    df_products.alias("p")
    .join(
        df_product_names.alias("pn"),
        col("p.product_name_id") == col("pn.product_name_id"),
        "left",
    )
    .join(
        df_product_categories.alias("pc"),
        col("p.product_category_id") == col("pc.product_category_id"),
        "left",
    )
    .join(
        df_sales.alias("s"),
        col("p.id") == col("s.sale_product_id"),
        "left_outer",
    )
    .select(
        col("pn.product_name"),
        col("pc.product_category"),
        col("p.rating").alias("product_rating_product_table"),
        col("p.reviews").alias("product_reviews_count_product_table"),
        col("s.sale_quantity"),
        col("s.sale_total_price"),
    )
)

product_aggregated_data = df_product_quality_base.groupBy(
    "product_name", "product_category"
).agg(
    avg("product_rating_product_table").alias("avg_product_rating"),
    avg("product_reviews_count_product_table").alias("total_reviews_count_avg"),
    sum("sale_quantity").alias("total_quantity_sold"),
    sum("sale_total_price").alias("total_revenue_from_product"),
)

product_aggregated_data = product_aggregated_data.withColumn(
    "total_reviews_count",
    col("total_reviews_count_avg")
)

product_rating_extremes_report_df = product_aggregated_data.select(
    "product_name",
    "product_category",
    "avg_product_rating",
    "total_reviews_count"
).orderBy(col("avg_product_rating").desc())

product_rating_extremes_report_df.write.jdbc(
    url=ch_jdbc_url,
    table="product_rating_extremes_report",
    mode="append",
    properties=ch_properties,
)

product_rating_extremes_report_df.toPandas().head()

,product_name,product_category,avg_product_rating,total_reviews_count
0,Cat Toy,Toy,3.080961,500.730033
1,Dog Food,Cage,3.058483,486.972027
2,Bird Cage,Food,3.024100,492.469912
3,Dog Food,Food,3.020548,500.684830
4,Bird Cage,Cage,3.003047,510.425986


In [27]:
product_sales_rating_correlation_report_df = product_aggregated_data.select(
    "product_name",
    "product_category",
    "avg_product_rating",
    "total_quantity_sold",
    "total_revenue_from_product"
)

product_sales_rating_correlation_report_df.write.jdbc(
    url=ch_jdbc_url,
    table="product_sales_rating_correlation_report",
    mode="append",
    properties=ch_properties,
)
product_sales_rating_correlation_report_df.toPandas().head()

,product_name,product_category,avg_product_rating,total_quantity_sold,total_revenue_from_product
0,Dog Food,Toy,2.997150,180267,8.388194e+06
1,Dog Food,Food,3.020548,171988,7.857492e+06
2,Dog Food,Cage,3.058483,162261,7.657714e+06
3,Bird Cage,Toy,2.995688,173390,8.339287e+06
4,Bird Cage,Cage,3.003047,177294,7.950889e+06


In [28]:
top_products_by_reviews_report_df = product_aggregated_data.select(
    "product_name",
    "product_category",
    "total_reviews_count"
).orderBy(col("total_reviews_count").desc())

top_products_by_reviews_report_df.write.jdbc(
    url=ch_jdbc_url,
    table="product_most_reviewed_report",
    mode="append",
    properties=ch_properties,
)
top_products_by_reviews_report_df.toPandas().head()

,product_name,product_category,total_reviews_count
0,Cat Toy,Food,519.717131
1,Bird Cage,Cage,510.425986
2,Cat Toy,Cage,506.695207
3,Dog Food,Toy,506.558436
4,Cat Toy,Toy,500.730033


In [29]:
spark.stop()